# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
filepath = 'data/song_data'

song_files = get_files(filepath)
song_file = song_files[0]
song_file

'/home/peter/syncthing/Ausbildung/Udacity/Data_Engineer/PyCharm/Project_1__Data_Modeling_with_Postgres/data/song_data/A/B/C/TRABCRU128F423F449.json'

In [5]:
df = pd.read_json(song_file, lines=True)
df.head()


,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR8IEZO1187B99055E,NaN,NaN,,Marc Shaiman,SOINLJW12A8C13314C,City Slickers,149.86404,2008


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [6]:
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0].tolist()
song_data

['SOINLJW12A8C13314C', 'City Slickers', 'AR8IEZO1187B99055E', 2008, 149.86404]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [7]:
song_table_insert = """INSERT INTO d_songs(song_id, title, artist_id, year, duration)
                     values(%s, %s, %s, %s, %s);"""
try:
    cur.execute(song_table_insert, song_data)
except psycopg2.Error as e:
    print("Error: Inserting d_songs data")
    print (e)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [8]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0].tolist()
artist_data



['AR8IEZO1187B99055E', 'Marc Shaiman', '', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [9]:
artist_table_insert = """INSERT INTO d_artists (artist_id, name, location, latitude, longitude)
                         values(%s, %s, %s, %s, %s);"""
try:
    cur.execute(artist_table_insert, artist_data)
except psycopg2.Error as e:
    print("Error: Inserting artist_data")
    print (e)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [10]:
filepath = 'data/log_data'

log_files = get_files(filepath)
log_file = log_files[0]
log_file

'/home/peter/syncthing/Ausbildung/Udacity/Data_Engineer/PyCharm/Project_1__Data_Modeling_with_Postgres/data/log_data/2018/11/2018-11-29-events.json'

In [11]:
df = pd.read_json(log_file, lines=True)
df.head()


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,1543449690796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,1543449841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
3,RÃÂ¶yksopp,Logged In,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,The Girl and The Robot,200,1543449895796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Kajagoogoo,Logged In,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,Too Shy,200,1543450033796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [12]:
# Filter records by `NextSong` action
df = df[df['page'] == 'NextSong']
df.head()


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,1543449690796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,1543449841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
3,RÃÂ¶yksopp,Logged In,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,The Girl and The Robot,200,1543449895796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Kajagoogoo,Logged In,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,Too Shy,200,1543450033796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


In [13]:
# Convert the `ts` timestamp column to datetime format --> this in only convertabile with the data frame
df_time = pd.to_datetime(df['ts'], unit='ms')

######### time_data = list of date_values"!!!"
## timestamp, hour, day, week of year, month, year, and weekday
df_time_timestamp = df_time
df_time_hour = df_time.dt.hour
df_time_day = df_time.dt.day
df_time_week_of_year = df_time.dt.weekofyear
df_time_month = df_time.dt.month
df_time_year = df_time.dt.year
df_time_weekday = df_time.dt.weekday

time_data = [df_time_timestamp, df_time_hour, df_time_day, df_time_week_of_year, df_time_month, df_time_year, df_time_weekday]
column_labels = ['start_time', 'hour', 'day', 'week_of_year', 'month', 'year', 'weekday']

# combine both lists together to key value pairs
#https://stackoverflow.com/questions/209840/convert-two-lists-into-a-dictionary
#>>> keys = ['a', 'b', 'c']
#>>> values = [1, 2, 3]
#>>> dictionary = dict(zip(keys, values))
#>>> print(dictionary)
#{'a': 1, 'b': 2, 'c': 3}

# combine column_lables together with time_data
dict(zip(column_labels, time_data))


{'start_time': 0     2018-11-29 00:00:57.796
 1     2018-11-29 00:01:30.796
 2     2018-11-29 00:04:01.796
 3     2018-11-29 00:04:55.796
 4     2018-11-29 00:07:13.796
                 ...          
 360   2018-11-29 22:53:32.796
 362   2018-11-29 22:57:22.796
 363   2018-11-29 23:02:05.796
 364   2018-11-29 23:05:13.796
 365   2018-11-29 23:18:35.796
 Name: ts, Length: 319, dtype: datetime64[ns],
 'hour': 0       0
 1       0
 2       0
 3       0
 4       0
        ..
 360    22
 362    22
 363    23
 364    23
 365    23
 Name: ts, Length: 319, dtype: int64,
 'day': 0      29
 1      29
 2      29
 3      29
 4      29
        ..
 360    29
 362    29
 363    29
 364    29
 365    29
 Name: ts, Length: 319, dtype: int64,
 'week_of_year': 0      48
 1      48
 2      48
 3      48
 4      48
        ..
 360    48
 362    48
 363    48
 364    48
 365    48
 Name: ts, Length: 319, dtype: int64,
 'month': 0      11
 1      11
 2      11
 3      11
 4      11
        ..
 360    11
 362

In [14]:
#build data_frame for time data
time_df = pd.DataFrame.from_dict(dict(zip(column_labels, time_data)), orient='columns',dtype=None)
time_df.head()

,start_time,hour,day,week_of_year,month,year,weekday
0,2018-11-29 00:00:57.796,0,29,48,11,2018,3
1,2018-11-29 00:01:30.796,0,29,48,11,2018,3
2,2018-11-29 00:04:01.796,0,29,48,11,2018,3
3,2018-11-29 00:04:55.796,0,29,48,11,2018,3
4,2018-11-29 00:07:13.796,0,29,48,11,2018,3


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [15]:
#convert the time_df into a list
time_df_list = time_df.values.tolist()

time_df_list

[[Timestamp('2018-11-29 00:00:57.796000'), 0, 29, 48, 11, 2018, 3],
 [Timestamp('2018-11-29 00:01:30.796000'), 0, 29, 48, 11, 2018, 3],
 [Timestamp('2018-11-29 00:04:01.796000'), 0, 29, 48, 11, 2018, 3],
 [Timestamp('2018-11-29 00:04:55.796000'), 0, 29, 48, 11, 2018, 3],
 [Timestamp('2018-11-29 00:07:13.796000'), 0, 29, 48, 11, 2018, 3],
 [Timestamp('2018-11-29 00:10:56.796000'), 0, 29, 48, 11, 2018, 3],
 [Timestamp('2018-11-29 00:11:04.796000'), 0, 29, 48, 11, 2018, 3],
 [Timestamp('2018-11-29 00:14:10.796000'), 0, 29, 48, 11, 2018, 3],
 [Timestamp('2018-11-29 00:16:02.796000'), 0, 29, 48, 11, 2018, 3],
 [Timestamp('2018-11-29 00:18:11.796000'), 0, 29, 48, 11, 2018, 3],
 [Timestamp('2018-11-29 00:22:13.796000'), 0, 29, 48, 11, 2018, 3],
 [Timestamp('2018-11-29 00:26:15.796000'), 0, 29, 48, 11, 2018, 3],
 [Timestamp('2018-11-29 00:30:53.796000'), 0, 29, 48, 11, 2018, 3],
 [Timestamp('2018-11-29 00:35:07.796000'), 0, 29, 48, 11, 2018, 3],
 [Timestamp('2018-11-29 00:40:14.796000'), 0, 29

In [16]:
time_table_insert = """INSERT INTO d_time (start_time, hour, day, week, month, year, weekday) 
                       values (%s, %s, %s, %s, %s, %s, %s)
                       ON CONFLICT (start_time) DO NOTHING;
                    """
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [17]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df


,userId,firstName,lastName,gender,level
0,73,Jacob,Klein,M,paid
1,24,Layla,Griffin,F,paid
2,24,Layla,Griffin,F,paid
3,73,Jacob,Klein,M,paid
4,24,Layla,Griffin,F,paid
...,...,...,...,...,...
360,80,Tegan,Levine,F,paid
362,80,Tegan,Levine,F,paid
363,80,Tegan,Levine,F,paid
364,80,Tegan,Levine,F,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [18]:
user_table_insert = ("""
INSERT INTO d_users (user_id, first_name, last_name, gender, level)
             values (%s, %s, %s, %s, %s)
             ON CONFLICT (user_id)
             DO UPDATE SET 
               (first_name, last_name, gender, level)=
               (EXCLUDED.first_name, EXCLUDED.last_name, EXCLUDED.gender, EXCLUDED.level)
               ;
""")

for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [19]:
#prepare timestamp from df['ts']
df_songplays_ts = pd.to_datetime(df['ts'], unit='ms')
df_songplays_ts


0     2018-11-29 00:00:57.796
1     2018-11-29 00:01:30.796
2     2018-11-29 00:04:01.796
3     2018-11-29 00:04:55.796
4     2018-11-29 00:07:13.796
                ...          
360   2018-11-29 22:53:32.796
362   2018-11-29 22:57:22.796
363   2018-11-29 23:02:05.796
364   2018-11-29 23:05:13.796
365   2018-11-29 23:18:35.796
Name: ts, Length: 319, dtype: datetime64[ns]

In [20]:
# preparing the select statement for song_id and artist_id
song_select = """
select s.song_id, a.artist_id
  from d_songs s
  join d_artists a on s.artist_id = a.artist_id
 where s.title = %s
   and a.name = %s
   and s.duration = %s
;
"""
cur.execute(song_select, ('Setanta matins', 'Elena', '269.58322'))
conn.commit()

results = cur.fetchone()
print(results)

None


In [21]:

#if results is true --> put result values into variables
if results: 
    songid, artistid = results
else: 
    songid, artistid = None, None

print(songid, artistid)


# prepare array songplay_data
for index, row in df.iterrows():
    songplay_data = [row.userId, songid, artistid, pd.to_datetime(row.ts, unit='ms'), row.sessionId, row.level, row.location, row.userAgent]
    print(songplay_data)
    

None None
['73', None, None, Timestamp('2018-11-29 00:00:57.796000'), 954, 'paid', 'Tampa-St. Petersburg-Clearwater, FL', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.78.2 (KHTML, like Gecko) Version/7.0.6 Safari/537.78.2"']
['24', None, None, Timestamp('2018-11-29 00:01:30.796000'), 984, 'paid', 'Lake Havasu City-Kingman, AZ', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"']
['24', None, None, Timestamp('2018-11-29 00:04:01.796000'), 984, 'paid', 'Lake Havasu City-Kingman, AZ', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"']
['73', None, None, Timestamp('2018-11-29 00:04:55.796000'), 954, 'paid', 'Tampa-St. Petersburg-Clearwater, FL', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.78.2 (KHTML, like Gecko) Version/7.0.6 Safari/537.78.2"']
['24', None, None, Timestamp('2018-11-29 00:07:13.796000'), 984, 'paid', 'Lake H

In [45]:
songplay_table_insert_INSERTintoSQL_QUERIES= """
INSERT INTO f_songplays (user_id, song_id, artist_id, start_time, session_id, level, location, user_agent)
            values (%s, %s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (songplay_id)
            DO NOTHING;
"""

for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
        cur.execute(song_select, (row.song, row.artist, row.length))

    # fetch all item out of cursor object into results object
        results = cur.fetchone()
        
    # if results is true --> put result values into variables
        if results:
            songid, artistid = results
        else:
            songid, artistid = None, None
 
    # insert songplay record
    songplay_data = [row.userId, songid, artistid, pd.to_datetime(row.ts, unit='ms'), row.sessionId, row.level, row.location, row.userAgent]
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()
print(songplay_data)



IndentationError: unindent does not match any outer indentation level (<tokenize>, line 24)

# Close Connection to Sparkify Database

In [46]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.